In [2]:
!nvidia-smi

Tue Jun  4 20:08:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# Dataset sent_aac
!gdown https://drive.google.com/uc?id=1DSGUQ-2ePO2HR7-ZwgY6VrR-TRn0ZGk_
!unzip multiple-features.zip

Downloading...
From: https://drive.google.com/uc?id=1DSGUQ-2ePO2HR7-ZwgY6VrR-TRn0ZGk_
To: /content/multiple-features.zip
100% 1.96M/1.96M [00:00<00:00, 16.9MB/s]
Archive:  multiple-features.zip
   creating: multiple-features/
  inflating: multiple-features/mfeat-fac  
  inflating: multiple-features/mfeat-fou  
  inflating: multiple-features/mfeat-zer  


In [3]:
import numpy as np

# Função para carregar os dados de um arquivo de texto
def load_data(filename):
    with open(filename, 'r') as file:
        data = np.loadtxt(file)
    return data

# Carregar os datasets mfeat-fou, mfeat-fac e mfeat-zer
mfeat_fou_data = load_data('multiple-features/mfeat-fou')
mfeat_fac_data = load_data('multiple-features/mfeat-fac')
mfeat_zer_data = load_data('multiple-features/mfeat-zer')

#print(mfeat_fou_data)
#print(mfeat_fac_data)
#print(mfeat_zer_data)

# Verificar as dimensões dos datasets carregados
print("Dimensões do dataset mfeat-fou:", mfeat_fou_data.shape)
print("Dimensões do dataset mfeat-fac:", mfeat_fac_data.shape)
print("Dimensões do dataset mfeat-zer:", mfeat_zer_data.shape)

Dimensões do dataset mfeat-fou: (2000, 76)
Dimensões do dataset mfeat-fac: (2000, 216)
Dimensões do dataset mfeat-zer: (2000, 47)


In [4]:
# Selecionar as primeiras 100 linhas de cada conjunto de dados
data1 = mfeat_fou_data
data2 = mfeat_fac_data
data3 = mfeat_zer_data
epsilon=1e-6

In [5]:
import numpy as np

min_value1 = np.min(data1)
max_value1 = np.max(data1)
mean_value1 = np.median(data1)

# Imprimir os resultados
print("Valor mínimo para mfeat_fou_data:", min_value1)
print("Valor máximo para mfeat_fou_data:", max_value1)
print("Valor da mediana para mfeat_fou_data:", mean_value1)

min_value2 = np.min(data2)
max_value2 = np.max(data2)
mean_value2 = np.median(data2)

# Imprimir os resultados
print("Valor mínimo para mfeat_fac_data:", min_value2)
print("Valor máximo para mfeat_fac_data:", max_value2)
print("Valor da mediana para mfeat_fac_data:", mean_value2)

min_value3 = np.min(data3)
max_value3 = np.max(data3)
mean_value3 = np.median(data3)

# Imprimir os resultados
print("Valor mínimo para mfeat_zer_data:", min_value3)
print("Valor máximo para mfeat_zer_data:", max_value3)
print("Valor da mediana para mfeat_zer_data:", mean_value3)


Valor mínimo para mfeat_fou_data: 0.00017629
Valor máximo para mfeat_fou_data: 0.79648722
Valor da mediana para mfeat_fou_data: 0.104986425
Valor mínimo para mfeat_fac_data: 0.0
Valor máximo para mfeat_fac_data: 1353.0
Valor da mediana para mfeat_fac_data: 53.0
Valor mínimo para mfeat_zer_data: 0.00109432
Valor máximo para mfeat_zer_data: 777.86182699
Valor da mediana para mfeat_zer_data: 42.03309036


In [6]:
import numpy as np

# Supondo que 'matriz' seja a sua matriz NumPy
np.set_printoptions(threshold=np.inf)

# **MFEAT-FOU**

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix, adjusted_rand_score
from scipy.special import comb

# Função para calcular o valor do kernel gaussiano de dados com centros de cluster
def gaussian_distance(data, centers, s):
    gaussian_dist = np.exp(-0.5 * np.sum(((data[:, np.newaxis] - centers) ** 2) / s**2, axis=2))
    return gaussian_dist

# Função para calcular os graus de pertinência usando o método lagrangeano
def update_membership_using_lagrangean(data, centers, s, m):
    gaussian_dist = gaussian_distance(data, centers, s)
    p = 2 - 2 * gaussian_dist
    p = np.fmax(p, np.finfo(np.float64).eps)
    p = (1/p) ** (1/(m-1))
    u_new = p / np.sum(p, axis=1, keepdims=True)
    u_new = np.fmax(u_new, np.finfo(np.float64).eps)
    return u_new

# Função para calcular os parâmetros de largura global usando o método lagrangeano
def update_width_parameters(data, U, gaussian_dist, m, centers):
    n_samples, n_features = data.shape
    s = np.zeros(n_features)
    um = U ** m

    for j in range(n_features):
        numerator = np.prod([
            np.sum([
                um[:, i] * gaussian_dist[:, i] * (data[:, h] - centers[i, h])**2
                for i in range(len(centers))
            ])
            for h in range(n_features)
        ])**(1/n_features)

        denominator = np.sum([
            um[:, i] * gaussian_dist[:, i] * (data[:, j] - centers[i, j])**2
            for i in range(len(centers))
        ])

        s[j] = numerator / denominator

    return s


# Função para calcular o Modified Partition Coefficient (MPC)
def calculate_mpc(U):
    n_samples, n_clusters = U.shape
    mpc = np.sum(U ** 2) / n_samples
    return mpc

# Função para calcular os centróides
def calculate_centers(data, U, m, gaussian_dist, n_clusters):
    centers = []
    um = U ** m
    for n in range(n_clusters):
        p = um[:, n].reshape(-1, 1) * gaussian_dist[:, n].reshape(-1, 1)
        q = p * data
        center_i = np.sum(q, axis=0) / np.sum(p, axis=0)
        centers.append(center_i)
    centers = np.array(centers)
    return centers

# Função para calcular J_{KFCM-W.1}
def compute_cost(U, m, gaussian_dist):
    cost = np.sum((U ** m) * (2 - 2 * gaussian_dist))
    return cost

# Função principal para executar o algoritmo KFCM-K-W.1
def gaussian_kernel_fuzzy_c_means_KW(data, n_clusters, m, max_iters, tol, random_state=None):
    # Inicialização dos parâmetros e variáveis
    if random_state is not None:
        np.random.seed(random_state)

    n_samples, n_features = data.shape
    U = np.random.rand(n_samples, n_clusters)
    U /= np.sum(U, axis=1, keepdims=True)
    centers = np.random.permutation(data)[:n_clusters]
    s = np.ones(n_features)

    for iteration in range(max_iters):
        # Salva os valores de associação fuzzy antes de atualizar
        U_old = U.copy()

        # Passo 1: Computa a distância gaussiana
        gaussian_dist = gaussian_distance(data, centers, s)

        # Passo 2: Atualiza os parâmetros de largura global
        s = update_width_parameters(data, U, gaussian_dist, m, centers)

        # Passo 3: Atualiza os centróides
        centers = update_cluster_prototypes(data, U, gaussian_dist, m)

        # Passo 4: Atualiza os valores de associação fuzzy
        U = update_membership_using_lagrangean(data, centers, s, m)

        # Calcula a diferença e verifica a convergência
        diff = np.linalg.norm(U - U_old)
        if diff < tol:
            break

    cost = compute_cost(U, m, gaussian_dist)
    return U, centers, s, cost


# Função para calcular os protótipos dos clusters fuzzy usando o método lagrangeano
def update_cluster_prototypes(data, U, gaussian_dist, m):
    n_samples, n_features = data.shape
    centers = []
    um = U ** m  # Definindo um corretamente

    for n in range(n_clusters):
        p = um[:, n].reshape(-1, 1) * gaussian_dist[:, n].reshape(-1, 1)
        q = p * data
        center_i = np.sum(q, axis=0) / np.sum(p, axis=0)
        centers.append(center_i)
    centers = np.array(centers)
    return centers


# Função para calcular os vetores de parâmetros de largura de cada grupo
def calculate_group_width_parameters(data, U, group_prototypes, m):
    s = []
    um = U ** m
    for i, prototype in enumerate(group_prototypes):
        dist = np.sum(um[:, i] * np.linalg.norm(data - prototype, axis=1))
        s.append(dist / np.sum(um[:, i]))
    return np.array(s)

# Função para criar a partição crisp baseada nos valores de pertinência
def create_crisp_partition1(U):
    return np.argmax(U, axis=1)

# Função para calcular o ARI
def calculate_ari(labels_true, labels_pred):
    return adjusted_rand_score(labels_true, labels_pred)

# Definir parâmetros
n_clusters = 10  # Número de clusters
m = 1.6  # Parâmetro de fuzziness
max_iters = 100  # Número máximo de iterações
tol = 1e-6  # Tolerância para a convergência
n_runs = 50  # Número de execuções

# Executar o algoritmo para o dataset data1
best_cost = float('inf')
best_U, best_centers, best_s = None, None, None

for run in range(n_runs):
    U, centers, s, cost = gaussian_kernel_fuzzy_c_means_KW(data1, n_clusters, m, max_iters, tol, random_state=run)
    if cost < best_cost:
        best_cost = cost
        best_U = U
        best_centers = centers
        best_s = s

# Verificar se um resultado válido foi encontrado
if best_U is None:
    print("\nNão foi possível encontrar uma solução válida para o dataset data1.")
else:
    # Calcular os protótipos de cada grupo
    U, centers, s, cost = gaussian_kernel_fuzzy_c_means_KW(data1, n_clusters, m, max_iters, tol, random_state=run)
    gaussian_dist = gaussian_distance(data1, centers, s)  # Capturar a distância gaussiana
    group_prototypes = update_cluster_prototypes(data1, best_U, gaussian_dist, m)  # Passar gaussian_dist aqui

    # Calcular os vetores de parâmetros de largura de cada grupo
    group_width_parameters = calculate_group_width_parameters(data1, best_U, group_prototypes, m)

    # Criar a partição crisp
    crisp_partition1 = create_crisp_partition1(best_U)

    # Calcular o Modified Partition Coefficient (MPC)
    mpc = calculate_mpc(best_U)

    # Calcular o ARI entre duas execuções diferentes do algoritmo (não supervisionado)
    # Executar uma segunda vez para obter outra partição dos clusters
    U2, centers2, s2, cost2 = gaussian_kernel_fuzzy_c_means_KW(data1, n_clusters, m, max_iters, tol, random_state=n_runs)
    class_cluster_list2 = create_crisp_partition1(U2)

    # Calcular o ARI
    ari = calculate_ari(crisp_partition1, class_cluster_list2)

    # Calcular a matriz de confusão entre a partição crisp e a partição a priori
    confusion_mat = confusion_matrix(crisp_partition1, class_cluster_list2)

    # Exibir os resultados
    print("\nResultados para o dataset mfeat-fou:")
    print("Melhor valor da função objetivo:", best_cost)
    print("Melhores centróides do cluster:")
    print(best_centers)
    print("Melhores parâmetros de largura global:")
    print(best_s)
    print("Modified Partition Coefficient (MPC):", mpc)
    print("Índice de Rand corrigido (ARI):", ari)
    print("Matriz de confusão entre a partição crisp e a primeira execução:")
    print(confusion_mat)



Resultados para o dataset mfeat-fou:
Melhor valor da função objetivo: 540.3114864073782
Melhores centróides do cluster:
[[0.05423684 0.17058398 0.08498476 0.28541313 0.5657523  0.03749419
  0.41461882 0.07555099 0.36622722 0.09836001 0.05479419 0.23046931
  0.06194344 0.34473739 0.1815538  0.04399275 0.13079252 0.07004204
  0.09730777 0.0832607  0.0579864  0.12053932 0.05457204 0.15074673
  0.10010536 0.05105171 0.08515712 0.06764164 0.08413526 0.06857186
  0.0583387  0.08019258 0.05586906 0.08741404 0.06856224 0.05348734
  0.06486561 0.06212489 0.06939111 0.05780052 0.05530069 0.06146115
  0.05438422 0.06314533 0.05351011 0.05875188 0.05326075 0.06626506
  0.05457638 0.06614284 0.06060796 0.06429509 0.05534138 0.07902967
  0.05501436 0.0918326  0.05393474 0.08871069 0.06277538 0.07840695
  0.05270398 0.11985142 0.05101061 0.15914397 0.04643432 0.13835149
  0.06209394 0.0895241  0.05012587 0.2247047  0.05109884 0.33876532
  0.03371741 0.414179   0.0679423  0.36976436]
 [0.16493713 0.5

# **MFEAT-FAC**

In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix, adjusted_rand_score
from scipy.special import comb

# Função para calcular o valor do kernel gaussiano de dados com centros de cluster
def gaussian_distance(data, centers, s):
    gaussian_dist = np.exp(-0.5 * np.sum(((data[:, np.newaxis] - centers) ** 2) / (s**2+epsilon), axis=2))
    return gaussian_dist

# Função para calcular os graus de pertinência usando o método lagrangeano
def update_membership_using_lagrangean(data, centers, s, m):
    gaussian_dist = gaussian_distance(data, centers, s)
    p = 2 - 2 * gaussian_dist
    p = np.fmax(p, np.finfo(np.float64).eps)
    p = (1/p) ** (1/(m-1))
    u_new = p / np.sum(p, axis=1, keepdims=True)
    u_new = np.fmax(u_new, np.finfo(np.float64).eps)
    return u_new

# Função para calcular os parâmetros de largura global usando o método lagrangeano
def update_width_parameters(data, U, gaussian_dist, m, centers):
    n_samples, n_features = data.shape
    s = np.zeros(n_features)
    um = U ** m
    epsilon = 1e-6  # Definindo epsilon antes de usá-lo

    for j in range(n_features):
        numerator = np.sum([
            np.sum([
                um[:, i] * gaussian_dist[:, i] * np.sum((data[:, j] - centers[i][j])**2)
                for i in range(len(centers))
            ])
        ])

        denominator = np.sum([
            np.sum([
                um[:, i] * gaussian_dist[:, i]
                for i in range(len(centers))
            ])
        ])

        s[j] = numerator / (denominator + epsilon)  # Adiciona epsilon ao denominador

    return s



# Função para calcular os centróides
def update_cluster_prototypes(data, U, gaussian_dist, m):
    n_samples, n_features = data.shape
    centers = []
    um = U ** m

    for n in range(len(um[0])):
        um_sum = np.sum(um[:, n])
        center_n = np.sum(data * um[:, n].reshape(-1, 1), axis=0) / um_sum
        centers.append(center_n)

    return np.array(centers)

# Função para calcular J_{KFCM-W.1}
def compute_cost(U, m, gaussian_dist):
    cost = np.sum((U ** m) * (2 - 2 * gaussian_dist))
    return cost

# Função principal para executar o algoritmo KFCM-K-W.1
def gaussian_kernel_fuzzy_c_means_KW(data, n_clusters, m, max_iters, tol, random_state=None):
    # Inicialização dos parâmetros e variáveis
    if random_state is not None:
        np.random.seed(random_state)

    n_samples, n_features = data.shape
    U = np.random.rand(n_samples, n_clusters)
    U /= np.sum(U, axis=1, keepdims=True)
    um = U ** m  # Definição correta de um
    centers = np.random.permutation(data)[:n_clusters]
    s = np.ones(n_features)

    for iteration in range(max_iters):
        # Salva os valores de associação fuzzy antes de atualizar
        U_old = U.copy()

        # Passo 1: Computa a distância gaussiana
        gaussian_dist = gaussian_distance(data, centers, s)

        # Passo 2: Atualiza os parâmetros de largura global
        s = update_width_parameters(data, U, gaussian_dist, m, centers)

        # Passo 3: Atualiza os centróides
        centers = update_cluster_prototypes(data, U, gaussian_dist, m)

        # Passo 4: Atualiza os valores de associação fuzzy
        U = update_membership_using_lagrangean(data, centers, s, m)

        # Calcula a diferença e verifica a convergência
        diff = np.linalg.norm(U - U_old)
        if diff < tol:
            break

    cost = compute_cost(U, m, gaussian_dist)
    return U, centers, s, cost





# Função para calcular o Modified Partition Coefficient (MPC)
def calculate_mpc(U):
    n_samples, n_clusters = U.shape
    mpc = np.sum(U ** 2) / n_samples
    return mpc

# Função para calcular o ARI
def calculate_ari(labels_true, labels_pred):
    return adjusted_rand_score(labels_true, labels_pred)

# Definir parâmetros
n_clusters = 10  # Número de clusters
m = 1.2  # Parâmetro de fuzziness
max_iters = 100  # Número máximo de iterações
tol = 1e-6  # Tolerância para a convergência
n_runs = 50  # Número de execuções

# Executar o algoritmo para o dataset data2
best_cost = float('inf')
best_U, best_centers, best_s = None, None, None

for run in range(n_runs):
    U, centers, s, cost = gaussian_kernel_fuzzy_c_means_KW(data2, n_clusters, m, max_iters, tol, random_state=run)
    if cost < best_cost:
        best_cost = cost
        best_U = U
        best_centers = centers
        best_s = s

# Verificar se um resultado válido foi encontrado
if best_U is None:
    print("\nNão foi possível encontrar uma solução válida para o dataset data2.")
else:
    # Calcular os protótipos de cada grupo
    U, centers, s, cost = gaussian_kernel_fuzzy_c_means_KW(data2, n_clusters, m, max_iters, tol, random_state=run)
    gaussian_dist = gaussian_distance(data2, centers, s)  # Capturar a distância gaussiana

    # Criar a partição crisp
    crisp_partition = np.argmax(best_U, axis=1)

    # Calcular o Modified Partition Coefficient (MPC)
    mpc = calculate_mpc(best_U)

    # Calcular o ARI entre duas execuções diferentes do algoritmo (não supervisionado)
    # Executar uma segunda vez para obter outra partição dos clusters
    U2, _, _, _ = gaussian_kernel_fuzzy_c_means_KW(data2, n_clusters, m, max_iters, tol, random_state=n_runs)
    class_cluster_list2 = np.argmax(U2, axis=1)

    # Calcular o ARI
    ari = calculate_ari(crisp_partition, class_cluster_list2)

    # Calcular a matriz de confusão entre a partição crisp e a partição a priori
    confusion_mat = confusion_matrix(crisp_partition, class_cluster_list2)

    # Exibir os resultados
    print("\nResultados para o dataset mfeat-fac:")
    print("Melhor valor da função objetivo:", best_cost)
    print("Melhores centróides do cluster:")
    print(best_centers)
    print("Melhores parâmetros de largura global:")
    print(best_s)
    print("Modified Partition Coefficient (MPC):", mpc)
    print("Índice de Rand corrigido (ARI):", ari)
    print("Matriz de confusão entre a partição crisp e a primeira execução:")
    print(confusion_mat)





Resultados para o dataset mfeat-fac:
Melhor valor da função objetivo: 0.0005546922714490711
Melhores centróides do cluster:
[[ 274.30259405  273.45039696  670.32606495  795.84471477  639.5953305
   658.27033043    9.81618578   15.82116987   16.08111544    9.44760111
    10.85148552    7.78725447  234.30317921  270.17746453  690.53261971
   709.23578583  635.45452017 1005.36166058    8.82737845   24.43138533
    17.2432554     5.62532284    9.32449765   16.8369173   259.84487164
   378.68767046  693.86799391  642.88272004  856.34624645  729.42962594
    27.257471     20.53366742    9.09442127    8.06311554    5.18388828
     9.93168544  357.23369113  341.83617574  760.53743785  810.88473305
   745.1875889   918.97796102   27.23026996   22.85044879   17.76733465
    11.15134977   13.40102909   11.28987911  313.88006296  362.54511526
   738.29764106  785.95695837  816.14113851  747.28519676   33.06518877
    24.1866386     9.09442127    9.38956991   14.12913826   13.10920341
   319.19563

# **MFEAT-ZER**

In [10]:
import numpy as np
from sklearn.metrics import confusion_matrix, adjusted_rand_score
from scipy.special import comb

# Função para calcular o valor do kernel gaussiano de dados com centros de cluster
def gaussian_distance(data, centers, s):
    gaussian_dist = np.exp(-0.5 * np.sum(((data[:, np.newaxis] - centers) ** 2) / (s**2+epsilon), axis=2))
    return gaussian_dist

# Função para calcular os graus de pertinência usando o método lagrangeano
def update_membership_using_lagrangean(data, centers, s, m):
    gaussian_dist = gaussian_distance(data, centers, s)
    p = 2 - 2 * gaussian_dist
    p = np.fmax(p, np.finfo(np.float64).eps)
    p = (1/p) ** (1/(m-1))
    u_new = p / np.sum(p, axis=1, keepdims=True)
    u_new = np.fmax(u_new, np.finfo(np.float64).eps)
    return u_new

# Função para calcular os parâmetros de largura global usando o método lagrangeano
def update_width_parameters(data, U, gaussian_dist, m, centers):
    n_samples, n_features = data.shape
    s = np.zeros(n_features)
    um = U ** m
    epsilon = 1e-6  # Definindo epsilon antes de usá-lo

    for j in range(n_features):
        numerator = np.sum([
            np.sum([
                um[:, i] * gaussian_dist[:, i] * np.sum((data[:, j] - centers[i][j])**2)
                for i in range(len(centers))
            ])
        ])

        denominator = np.sum([
            np.sum([
                um[:, i] * gaussian_dist[:, i]
                for i in range(len(centers))
            ])
        ])

        s[j] = numerator / (denominator + epsilon)  # Adiciona epsilon ao denominador

    return s



# Função para calcular os centróides
def update_cluster_prototypes(data, U, gaussian_dist, m):
    n_samples, n_features = data.shape
    centers = []
    um = U ** m

    for n in range(len(um[0])):
        um_sum = np.sum(um[:, n])
        center_n = np.sum(data * um[:, n].reshape(-1, 1), axis=0) / um_sum
        centers.append(center_n)

    return np.array(centers)

# Função para calcular J_{KFCM-W.1}
def compute_cost(U, m, gaussian_dist):
    cost = np.sum((U ** m) * (2 - 2 * gaussian_dist))
    return cost

# Função principal para executar o algoritmo KFCM-K-W.1
def gaussian_kernel_fuzzy_c_means_KW(data, n_clusters, m, max_iters, tol, random_state=None):
    # Inicialização dos parâmetros e variáveis
    if random_state is not None:
        np.random.seed(random_state)

    n_samples, n_features = data.shape
    U = np.random.rand(n_samples, n_clusters)
    U /= np.sum(U, axis=1, keepdims=True)
    um = U ** m  # Definição correta de um
    centers = np.random.permutation(data)[:n_clusters]
    s = np.ones(n_features)

    for iteration in range(max_iters):
        # Salva os valores de associação fuzzy antes de atualizar
        U_old = U.copy()

        # Passo 1: Computa a distância gaussiana
        gaussian_dist = gaussian_distance(data, centers, s)

        # Passo 2: Atualiza os parâmetros de largura global
        s = update_width_parameters(data, U, gaussian_dist, m, centers)

        # Passo 3: Atualiza os centróides
        centers = update_cluster_prototypes(data, U, gaussian_dist, m)

        # Passo 4: Atualiza os valores de associação fuzzy
        U = update_membership_using_lagrangean(data, centers, s, m)

        # Calcula a diferença e verifica a convergência
        diff = np.linalg.norm(U - U_old)
        if diff < tol:
            break

    cost = compute_cost(U, m, gaussian_dist)
    return U, centers, s, cost





# Função para calcular o Modified Partition Coefficient (MPC)
def calculate_mpc(U):
    n_samples, n_clusters = U.shape
    mpc = np.sum(U ** 2) / n_samples
    return mpc

# Função para calcular o ARI
def calculate_ari(labels_true, labels_pred):
    return adjusted_rand_score(labels_true, labels_pred)

# Definir parâmetros
n_clusters = 10  # Número de clusters
m = 1.6  # Parâmetro de fuzziness
max_iters = 100  # Número máximo de iterações
tol = 1e-6  # Tolerância para a convergência
n_runs = 50  # Número de execuções

# Executar o algoritmo para o dataset data2
best_cost = float('inf')
best_U, best_centers, best_s = None, None, None

for run in range(n_runs):
    U, centers, s, cost = gaussian_kernel_fuzzy_c_means_KW(data3, n_clusters, m, max_iters, tol, random_state=run)
    if cost < best_cost:
        best_cost = cost
        best_U = U
        best_centers = centers
        best_s = s

# Verificar se um resultado válido foi encontrado
if best_U is None:
    print("\nNão foi possível encontrar uma solução válida para o dataset data2.")
else:
    # Calcular os protótipos de cada grupo
    U, centers, s, cost = gaussian_kernel_fuzzy_c_means_KW(data3, n_clusters, m, max_iters, tol, random_state=run)
    gaussian_dist = gaussian_distance(data3, centers, s)  # Capturar a distância gaussiana

    # Criar a partição crisp
    crisp_partition = np.argmax(best_U, axis=1)

    # Calcular o Modified Partition Coefficient (MPC)
    mpc = calculate_mpc(best_U)

    # Calcular o ARI entre duas execuções diferentes do algoritmo (não supervisionado)
    # Executar uma segunda vez para obter outra partição dos clusters
    U2, _, _, _ = gaussian_kernel_fuzzy_c_means_KW(data3, n_clusters, m, max_iters, tol, random_state=n_runs)
    class_cluster_list2 = np.argmax(U2, axis=1)

    # Calcular o ARI
    ari = calculate_ari(crisp_partition, class_cluster_list2)

    # Calcular a matriz de confusão entre a partição crisp e a partição a priori
    confusion_mat = confusion_matrix(crisp_partition, class_cluster_list2)

    # Exibir os resultados
    print("\nResultados para o dataset mfeat-zer:")
    print("Melhor valor da função objetivo:", best_cost)
    print("Melhores centróides do cluster:")
    print(best_centers)
    print("Melhores parâmetros de largura global:")
    print(best_s)
    print("Modified Partition Coefficient (MPC):", mpc)
    print("Índice de Rand corrigido (ARI):", ari)
    print("Matriz de confusão entre a partição crisp e a primeira execução:")
    print(confusion_mat)





Resultados para o dataset mfeat-zer:
Melhor valor da função objetivo: 0.009300065718004335
Melhores centróides do cluster:
[[2.89688800e-01 3.14967816e+00 3.78824429e+01 1.06046822e+02
  1.38326604e+02 1.49635917e+02 2.10611006e+02 4.00864331e-01
  4.46153658e+00 6.05349368e+01 1.34909425e+02 1.36445725e+02
  1.46701529e+02 4.04883628e-01 1.23976258e+01 7.47319726e+01
  1.42950996e+02 1.23889087e+02 1.33762364e+02 6.16362821e-01
  2.14210223e+01 9.70440479e+01 1.03775291e+02 1.99305723e+02
  1.89264761e+00 3.05998423e+01 9.84065235e+01 2.39030860e+02
  8.86918984e+01 3.53840324e+00 4.20687455e+01 7.36055199e+01
  1.59251173e+02 5.70921313e+00 4.19815426e+01 3.29507206e+02
  7.64769365e+01 8.44424486e+00 3.97698696e+01 1.31885791e+02
  8.54968834e+00 2.31861073e+02 2.73648026e+02 1.03688259e+01
  1.33239023e+02 6.98657610e+01 4.29420838e+02]
 [1.01647041e-01 1.92812423e+00 2.45728804e+01 7.76375243e+01
  1.13094383e+02 2.25758239e+02 1.53731753e+02 1.81106790e-01
  3.86490856e+00 3.007

# **NOVAS TENTATIVAS**